In [99]:
from adam.core.computations import KinDynComputations
from adam.geometry import utils
import numpy as np
import casadi as cs
import math
import gym_ignition_models
from adam.core.computationsWithHardwareParameters import  KinDynComputationsWithHardwareParameters
from adam.geometry import linkParametric
from urdfpy import xyz_rpy_to_matrix, matrix_to_xyz_rpy

In [100]:
def SX2DM(sx):
    return cs.DM(sx)

# Loading the model 

In [101]:

urdf_path ="/home/carlotta/iit_ws/ergocub-gazebo-simulations/models/stickBot/model.urdf"
urdf_path_old = gym_ignition_models.get_model_file("iCubGazeboV2_5")
# The joint list
joints_name_list = [
    'torso_pitch', 'torso_roll', 'torso_yaw', 'l_shoulder_pitch',
    'l_shoulder_roll', 'l_shoulder_yaw', 'l_elbow', 'r_shoulder_pitch',
    'r_shoulder_roll', 'r_shoulder_yaw', 'r_elbow', 'l_hip_pitch', 'l_hip_roll',
    'l_hip_yaw', 'l_knee', 'l_ankle_pitch', 'l_ankle_roll', 'r_hip_pitch',
    'r_hip_roll', 'r_hip_yaw', 'r_knee', 'r_ankle_pitch', 'r_ankle_roll'
]
link_name_list = ['l_lower_leg']
link_name_list_empty  = []
r_hip_r_char = linkParametric.LinkCharacteristics(-0.03)
r_hip_yaw_char = linkParametric.JointCharacteristics(-0.055989)
link_name = 'l_lower_leg'
link_characteristics = {link_name: r_hip_r_char}
joint_characteristics = {'l_ankle_pitch': r_hip_yaw_char}
print(joint_characteristics)
root_link = 'root_link'
kinDyn =KinDynComputationsWithHardwareParameters(urdf_path, joints_name_list,link_name_list, root_link, link_characteristics, joint_characteristics)

kinDynNoParametric = KinDynComputations(urdf_path,joints_name_list, root_link)
kynDynParam_NoLink = KinDynComputationsWithHardwareParameters(urdf_path, joints_name_list, link_name_list_empty, root_link)


{'l_ankle_pitch': <adam.geometry.linkParametric.JointCharacteristics object at 0x7fe3a4b06850>}


Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor


### Variable definition

In [102]:
# Set joints and base informations
link_original = kinDyn.get_element_by_name(link_name, kinDyn.robot)
r_hip_r_char = linkParametric.LinkCharacteristics(0.058)
length = link_original.visuals[0].geometry.cylinder.length 
radius = link_original.visuals[0].geometry.cylinder.radius
volume = math.pi*radius**2*length
mass = link_original.inertial.mass
density_num = mass/volume

xyz = (np.random.rand(3) - 0.5) * 5
rpy = (np.random.rand(3) - 0.5) * 5
H_b = utils.H_from_PosRPY(xyz, rpy)
v_b = (np.random.rand(6) - 0.5) * 5
s = (np.random.rand(len(joints_name_list)) - 0.5) * 5
s_dot = (np.random.rand(len(joints_name_list)) - 0.5) * 5
density =density_num 
lenght_multiplier = 1
gravity =np.array([0, 0, -9.80665, 0, 0, 0])

### Mass and Centroidal Matrix 

In [103]:
[M, J_cmm] = kinDyn.crba()
M = kinDyn.mass_matrix_fun() 
J_cmm = kinDyn.centroidal_momentum_matrix_fun()

# print("M", M(H_b, s, density, lenght_multiplier))
# print("J_cmm",J_cmm(H_b, s, density, lenght_multiplier))

In [104]:
M_no_parametric = kinDynNoParametric.mass_matrix_fun()
M_parametric_no_link = kynDynParam_NoLink.mass_matrix_fun()
# M_no_parametric_value = M_no_parametric(H_b, s)
M_no_parametric_value = M_parametric_no_link(H_b,s,density,lenght_multiplier)
M_value = M(H_b, s, density, lenght_multiplier)
# print("M", M_no_parametric(H_b, s,density, lenght_multiplier))



In [105]:
error = 0 
for i in range(29):
    for j in range(29):
        err = float(M_no_parametric_value[i,j] - M_value[i,j])
        error += abs(err)
print('Total Error Mass Matrices', error)

Total Error Mass Matrices 0.0


In [134]:
J_cmm_no_parameric = kinDynNoParametric.centroidal_momentum_matrix_fun()
# J_cmm_no_parameric_value = J_cmm_no_parameric(H_b,s)
J_cmm_parametric_no_link = kynDynParam_NoLink.centroidal_momentum_matrix_fun()
J_cmm_no_parameric_value = J_cmm_parametric_no_link(H_b,s,density,lenght_multiplier)
J_cmm_value = J_cmm(H_b,s,density, lenght_multiplier)
error = 0 
for i in range(6):
    for j in range (29): 
        err = float(J_cmm_no_parameric_value[i,j] - J_cmm_value[i,j])
        error += abs(err)
print("Total error centroidal momentum matrix", error)



Total error centroidal momentum matrix 0.0


### Total mass

In [135]:
mass = kinDyn.get_total_mass()
print('mass:\n',mass(density, lenght_multiplier))

mass:
 62.5739


In [190]:
# mass_no_param = kinDynNoParametric.get_total_mass()
mass_param_no_link = kynDynParam_NoLink.get_total_mass()
mass_no_param = mass_param_no_link(density,lenght_multiplier)
mass = kinDyn.get_total_mass()
mass_value = mass(density, lenght_multiplier)
error = float(mass_no_param-mass_value)
print("error mass", error)

error mass 0.0


### Forward Kinematics

In [191]:
H = kinDyn.forward_kinematics_fun('l_lower_leg')
print('H left sole :\n', H(H_b, s, density, lenght_multiplier))

H left sole :
 @1=0, 
[[0.604155, 0.152579, -0.782123, 1.09919], 
 [-0.38145, 0.917115, -0.115739, -0.0121536], 
 [0.699638, 0.368266, 0.612281, -1.2981], 
 [@1, @1, @1, 1]]


In [216]:
H_no_param = kinDynNoParametric.forward_kinematics_fun('l_lower_leg')
H_link_no_param = kynDynParam_NoLink.forward_kinematics_fun('l_lower_leg')
# H_no_param_value = H_no_param(H_b, s)
H_no_param_value = H_link_no_param(H_b,s,density,lenght_multiplier)
H_value = H(H_b, s, density,lenght_multiplier)
error = 0 
for i in range(4):
    for j in range(4):
        err = float(H_value[i,j]- H_no_param_value[i,j])
        error+= abs(err)

print("error forward kinematic", error)

error forward kinematic 0.0


### Center of Mass

In [217]:
com = kinDyn.CoM_position_fun()
print('CoM:\n',com(H_b, s, density, lenght_multiplier))

CoM:
 [1.02654, -0.0291284, -1.53071]


### Jacobian

In [278]:
CoM_no_param = kinDynNoParametric.CoM_position_fun()
CoM_param_no_link = kynDynParam_NoLink.CoM_position_fun()

# CoM_no_param_value = CoM_no_param(H_b,s)
CoM_no_param_value = CoM_param_no_link(H_b,s,density, lenght_multiplier)
CoM_value = com(H_b, s, density, lenght_multiplier)
error = 0
for i in range(3):
    err = float(CoM_no_param_value[i] - CoM_value[i])
    error+= abs(err)

print("error CoM", error)

error CoM 0.0


In [279]:
J = kinDyn.jacobian_fun('l_sole')
print('Jacobian left sole\n',J(H_b, s, density, lenght_multiplier))

Jacobian left sole
 @1=1, @2=0, 
[[@1, @2, @2, @2, 0.168588, 0.053836, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.131508, 0.108725, -0.0145113, 0.0951515, 0.0595055, -0.011769, @2, @2, @2, @2, @2, @2], 
 [@2, @1, @2, -0.168588, @2, 0.389853, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, 0.0426534, 0.246741, -0.2594, -0.118481, 0.0208336, 0.0629777, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @1, -0.053836, -0.389853, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, 0.327967, -0.0843164, -0.111908, 0.255637, -0.0765374, 0.0149981, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @2, @1, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.577216, -0.767053, -0.239715, -0.152579, 0.152579, -0.402712, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @2, @2, @1, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.806729, 0.482501, 0.395845, -0.917115, 0.917115, -0.282593, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @2, @2, @2, @1, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.126533, 0.422875, -0.886478, -0.368266, 0.368266, 0.

In [280]:
J_no_param = kinDynNoParametric.jacobian_fun('l_sole')
J_param_no_link = kynDynParam_NoLink.jacobian_fun('l_sole')
J_no_param_value = J_param_no_link(H_b, s,density,lenght_multiplier)
# J_no_param_value = J_no_param(H_b, s)
J_value = J(H_b,s, density, lenght_multiplier)
error = 0 
for i in range(6): 
    for j in range(29): 
        err = float(J_no_param_value[i,j]- J_value[i,j])
        error+ - abs(err)

print("error Jacobian", error)

error Jacobian 0


### Relative Jacobian

In [281]:
J_r = kinDyn.relative_jacobian_fun('l_sole')
print('Jacobian left sole relative: ', J_r(s,density, lenght_multiplier))

Jacobian left sole relative:  
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0799304, -0.11198, 0.0669841, 0.267631, -0.0171278, -0.0327194, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.251142, -0.231802, -0.0083125, 0.0414701, 0.0459104, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.346824, -0.0648057, -0.14766, 0.129408, -0.0884296, 0.033931, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.500894, -0.863458, 0.188829, -0.188829, 0.391118, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0.0688036, -0.874532, 0.874532, -0.350267, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.865509, -0.499707, -0.446696, 0.446696, 0.851081, 0, 0, 0, 0, 0, 0]]


In [282]:
J_r_no_param = kinDynNoParametric.relative_jacobian_fun('l_sole')
J_r_param_no_link = kynDynParam_NoLink.relative_jacobian_fun('l_sole')
# J_r_no_param_value = J_r_no_param(s)
J_r_no_param_value = J_r_param_no_link(s,density,lenght_multiplier)
J_value = J_r(s, density, lenght_multiplier)
error = 0 
for i in range(6): 
    for j in range(23):
        err = float(J_value[i,j] - J_r_no_param_value[i,j])
        error += abs(err)

print("error relative jacobian", error)

error relative jacobian 0.0


In [283]:
tau = kinDyn.rnea()
print('tau: \n', tau(H_b, s, v_b, s_dot, gravity, density, lenght_multiplier))

tau: 
 [-7.13691, 3.7533, 574.259, -37.172, -25.0059, -7.87716, 1.26442, 33.6507, -5.56242, 0.399048, -2.13561, -0.191787, -1.14309, 0.495309, 0.528967, 0.0343391, 0.947464, 17.057, -3.80483, -8.5189, 6.82618, -0.409421, -1.12478, -11.9652, 18.6853, 4.62383, -4.10115, 6.11813, -0.702574]


In [300]:
tau_no_parametric = kinDynNoParametric.rnea()
tau_parametric_no_link =kynDynParam_NoLink.rnea()
# tau_no_parametric_value = tau_no_parametric(H_b, s, v_b, s_dot, gravity)
tau_no_parametric_value = tau_parametric_no_link(H_b,s, v_b, s_dot, gravity,density,lenght_multiplier)
tau_value = tau(H_b, s, v_b, s_dot, gravity, density,  lenght_multiplier)

error = 0 
for i in range(29):
    err = float (tau_value[i] - tau_no_parametric_value[i])
    error += abs(err)

print("tau error", error)

tau error 0.0


In [301]:
h = kinDyn.bias_force_fun()
print('Bias force term:\n', SX2DM(h(H_b, s, v_b, s_dot, density, lenght_multiplier)))

Bias force term:
 [-7.13691, 3.7533, 574.259, -37.172, -25.0059, -7.87716, 1.26442, 33.6507, -5.56242, 0.399048, -2.13561, -0.191787, -1.14309, 0.495309, 0.528967, 0.0343391, 0.947464, 17.057, -3.80483, -8.5189, 6.82618, -0.409421, -1.12478, -11.9652, 18.6853, 4.62383, -4.10115, 6.11813, -0.702574]


In [316]:
h_no_param = kinDynNoParametric.bias_force_fun()
H_param_no_link= kynDynParam_NoLink.bias_force_fun()
h_no_param_values = H_param_no_link(H_b,s,v_b,s_dot,density,lenght_multiplier)
# h_no_param_values = h_no_param(H_b,s,v_b,s_dot)
h_value = h(H_b,s,v_b,s_dot,density,lenght_multiplier)
error = 0
for i in range(29): 
    err = float(h_value[i]- h_no_param_values[i])
    error += abs(err)
print("bias forces error", error)

bias forces error 0.0


In [317]:
C = kinDyn.coriolis_term_fun()
print('Coriolis force term:\n', SX2DM(C(H_b, s, v_b, s_dot, density, lenght_multiplier)))

Coriolis force term:
 [-7.13691, 3.7533, -39.3808, 6.5126, -1.44435, -7.87716, -10.1086, -1.11528, 0.858614, 0.0655543, 1.28163, 0.141311, -0.798583, -0.882418, -1.17788, 0.422703, -0.517834, -5.8055, -3.74463, -3.56621, -3.96315, 1.07967, -1.35971, 2.61041, -1.77222, 2.14227, -5.85181, 5.61524, -1.13751]


In [330]:
C_no_param = kinDynNoParametric.coriolis_term_fun()
C_param_no_link = kynDynParam_NoLink.coriolis_term_fun()
C_no_param_value = C_param_no_link(H_b,s,v_b,s_dot,density,lenght_multiplier)
# C_no_param_value = C_no_param(H_b,s,v_b, s_dot)
C_value = C(H_b, s,v_b,s_dot,density,lenght_multiplier)
error = 0 
for i in range(29): 
    err = float(C_no_param_value[i] - C_value[i])
    error+= abs(err)
print("Coriolis error", error)

Coriolis error 0.0


In [331]:
G = kinDyn.gravity_term_fun()
print('Gravity term:\n', SX2DM(G(H_b, s, density, lenght_multiplier)))

Gravity term:
 [0, 1.42109e-14, 613.64, -43.6846, -23.5615, -5.32907e-15, 11.373, 34.7659, -6.42103, 0.333494, -3.41724, -0.333098, -0.344505, 1.37773, 1.70685, -0.388363, 1.4653, 22.8625, -0.0602015, -4.95269, 10.7893, -1.48909, 0.234925, -14.5756, 20.4575, 2.48156, 1.75066, 0.50289, 0.434934]


In [342]:
G_no_param = kinDynNoParametric.gravity_term_fun()
G_param_no_link = kynDynParam_NoLink.gravity_term_fun()
# G_no_param_value = G_no_param(H_b,s)
G_no_param_value = G_param_no_link(H_b,s,density,lenght_multiplier)
G_value = G(H_b,s,density,lenght_multiplier)
error = 0 
for i in range(29): 
    err = float(G_no_param_value[i]-G_value[i])
    error += abs(err)
print("gravity term error", error)

gravity term error 0.0


### Cylinder Link 

In [343]:
link_original = kinDyn.get_element_by_name('l_lower_leg', kinDyn.robot)
r_hip_r_char = linkParametric.LinkCharacteristics(-0.03)
print(link_original.visuals[0].geometry.cylinder)
length = link_original.visuals[0].geometry.cylinder.length 
radius = link_original.visuals[0].geometry.cylinder.radius
volume = math.pi*radius**2*length
mass = link_original.inertial.mass
density_num = mass/volume
print('****************ORIGINAL LINK******************')
print('radius=', radius)
print('length=', length)
print('origin=',matrix_to_xyz_rpy(link_original.inertial.origin))
print('volume=', volume)
print('mass=', mass)
print('density=', density_num)
print('inertia xx=', link_original.inertial.inertia[0,0])
print('inertia yy=', link_original.inertial.inertia[1,1])
print('inertia zz=', link_original.inertial.inertia[2,2])
link_parametric =linkParametric.linkParametric('l_lower_leg', 1.0,density_num, kinDyn.robot, link_original, r_hip_r_char)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new[1])
print('length=', link_parametric.visual_data_new[0])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)


****************ORIGINAL LINK******************
radius= 0.065
length= 0.2
origin= [-0.02       -0.015      -0.13        0.          0.         -1.57079633]
volume= 0.002654645792283376
mass= 5.66584
density= 2134.3111071426842
inertia xx= 0.02487067683333334
inertia yy= 0.02487067683333334
inertia zz= 0.011969087000000002
****************PARAMETRIC LINK****************
radius= 0.065
length= 0.2
origin= @1=0, [-0.02, -0.015, -0.13, @1, @1, -1.5708]
volume= 0.00265465
mass= 5.66584
density= 2134.3111071426842
inertia xx= 0.0248707
inertia yy= 0.0248707
inertia zz= 0.0119691


### Cylinder Link Parametric

In [344]:
rho = cs.SX.sym('rho', 1)
l_m = cs.SX.sym('l_m',1)
link_parametric =linkParametric.linkParametric('l_lower_leg', l_m,rho, kinDyn.robot, link_original, r_hip_r_char)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new[1])
print('length=', link_parametric.visual_data_new[0])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)

****************PARAMETRIC LINK****************
radius= 0.065
length= (0.2*l_m)
origin= @1=0, [-0.02, -0.015, (-0.03-((0.2*l_m)/2)), @1, @1, -1.5708]
volume= (0.0132732*(0.2*l_m))
mass= ((0.0132732*(0.2*l_m))*rho)
density= rho
inertia xx= @1=(0.2*l_m), (((0.0132732*@1)*rho)*((0.012675+sq(@1))/12))
inertia yy= @1=(0.2*l_m), (((0.0132732*@1)*rho)*((0.012675+sq(@1))/12))
inertia zz= ((0.004225*((0.0132732*(0.2*l_m))*rho))/2)


### Box Link 

In [345]:
link_original = kinDyn.get_element_by_name('chest', kinDyn.robot)
chest_char = linkParametric.LinkCharacteristics(0.0, linkParametric.Side.DEPTH, False, False)

width = link_original.visuals[0].geometry.box.size[0]
depth = link_original.visuals[0].geometry.box.size[2]
height = link_original.visuals[0].geometry.box.size[1]

volume = width*depth*height
mass = link_original.inertial.mass
density_num = mass/volume
print('****************ORIGINAL LINK******************')
print('width=', width)
print('depth=', depth)
print('height=', height)
print('origin=',matrix_to_xyz_rpy(link_original.inertial.origin))
print('volume=', volume)
print('mass=', mass)
print('density=', density_num)
print('inertia xx=', link_original.inertial.inertia[0,0])
print('inertia yy=', link_original.inertial.inertia[1,1])
print('inertia zz=', link_original.inertial.inertia[2,2])

link_parametric =linkParametric.linkParametric('chest', 1.0,density_num, kinDyn.robot, link_original, chest_char)
print('****************PARAMETRIC LINK****************')
print('width=', link_parametric.visual_data_new[0])
print('depth=', link_parametric.visual_data_new[2])
print('height=', link_parametric.visual_data_new[1])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)


****************ORIGINAL LINK******************
width= 0.2041
depth= 0.2092
height= 0.2152
origin= [ 0.          0.          0.091388    0.          0.         -1.57079633]
volume= 0.009188549344000001
mass= 5.75757
density= 626.6027187152905
inertia xx= 0.0432180860748
inertia yy= 0.04098499815637501
inertia zz= 0.042206754510375005
****************PARAMETRIC LINK****************
width= 0.2041
depth= 0.2092
height= 0.2152
origin= [ 0.          0.          0.091388    0.          0.         -1.57079633]
volume= 0.00918855
mass= 5.75757
density= 626.6027187152905
inertia xx= 0.0432181
inertia yy= 0.040985
inertia zz= 0.0422068


### Box Link Parametric

In [346]:
rho = cs.SX.sym('rho', 1)
l_m = cs.SX.sym('l_m',1)
link_parametric =linkParametric.linkParametric('chest', l_m,rho, kinDyn.robot, link_original, chest_char)
print('****************PARAMETRIC LINK****************')
print('width=', link_parametric.visual_data_new[0])
print('depth=', link_parametric.visual_data_new[2])
print('height=', link_parametric.visual_data_new[1])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)

****************PARAMETRIC LINK****************
width= 0.2041
depth= (0.2092*l_m)
height= 0.2152
origin= [ 0.          0.          0.091388    0.          0.         -1.57079633]
volume= (0.0439223*(0.2092*l_m))
mass= ((0.0439223*(0.2092*l_m))*rho)
density= rho
inertia xx= @1=(0.2092*l_m), ((((0.0439223*@1)*rho)*(0.046311+sq(@1)))/12)
inertia yy= @1=(0.2092*l_m), ((((0.0439223*@1)*rho)*(0.0416568+sq(@1)))/12)
inertia zz= ((0.0879679*((0.0439223*(0.2092*l_m))*rho))/12)


### Sphere Link

In [347]:
link_original = kinDyn.get_element_by_name('l_elbow_1', kinDyn.robot)

radius = link_original.visuals[0].geometry.sphere.radius

volume = 4*(math.pi*radius**3)/3
mass = link_original.inertial.mass
density_num = mass/volume
print('****************ORIGINAL LINK******************')
print('radius=', radius)
print('origin=',matrix_to_xyz_rpy(link_original.inertial.origin))
print('volume=', volume)
print('mass=', mass)
print('density=', density_num)
print('inertia xx=', link_original.inertial.inertia[0,0])
print('inertia yy=', link_original.inertial.inertia[1,1])
print('inertia zz=', link_original.inertial.inertia[2,2])

link_parametric =linkParametric.linkParametric('l_elbow_1', 1.0,density_num, kinDyn.robot, link_original, chest_char)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new)
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)

****************ORIGINAL LINK******************
radius= 0.04
origin= [-0.02        0.         -0.02       -0.07156449 -0.06196999 -1.29535745]
volume= 0.00026808257310632905
mass= 0.230254
density= 858.892084375342
inertia xx= 0.01
inertia yy= 0.01
inertia zz= 0.01
****************PARAMETRIC LINK****************
radius= 0.04
origin= [-0.02        0.         -0.02       -0.07156449 -0.06196999 -1.29535745]
volume= 0.00026808257310632905
mass= 0.230254
density= 858.892084375342
inertia xx= 0.00014736256
inertia yy= 0.00014736256
inertia zz= 0.00014736256


### Sphere Link Parametric 

In [348]:
rho = cs.SX.sym('rho', 1)
l_m = cs.SX.sym('l_m',1)
link_original = kinDyn.get_element_by_name('l_elbow_1', kinDyn.robot)
link_parametric =linkParametric.linkParametric('l_elbow_1', l_m,rho, kinDyn.robot, link_original)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new)
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)


****************PARAMETRIC LINK****************
radius= (0.04*l_m)
origin= [-0.02        0.         -0.02       -0.07156449 -0.06196999 -1.29535745]
volume= @1=(0.04*l_m), ((12.5664*(@1*sq(@1)))/3)
mass= @1=(0.04*l_m), (((12.5664*(@1*sq(@1)))/3)*rho)
density= rho
inertia xx= @1=(0.04*l_m), (((2*(((12.5664*(@1*sq(@1)))/3)*rho))*sq(@1))/5)
inertia yy= @1=(0.04*l_m), (((2*(((12.5664*(@1*sq(@1)))/3)*rho))*sq(@1))/5)
inertia zz= @1=(0.04*l_m), (((2*(((12.5664*(@1*sq(@1)))/3)*rho))*sq(@1))/5)


In [349]:
r_hip_r_char = linkParametric.LinkCharacteristics(-0.03)
r_hip_yaw_char = linkParametric.JointCharacteristics(-0.055989)
joint_original = kinDyn.get_joint_by_name('l_ankle_pitch',kinDyn.robot)
link_original = kinDyn.get_element_by_name('l_lower_leg', kinDyn.robot)
link_parametric =linkParametric.linkParametric('l_lower_leg', l_m,rho, kinDyn.robot, link_original, r_hip_r_char)
joint_parametric = linkParametric.jointParametric('l_ankle_pitch', link_parametric, joint_original, r_hip_yaw_char)
print(matrix_to_xyz_rpy(joint_original.origin))
print(joint_parametric.origin)

[ 0.       -0.0005   -0.255989  0.        0.        0.      ]
@1=0, [@1, -0.0005, (-0.055989-(0.2*l_m)), @1, @1, @1]


In [350]:
Sc = cs.SX.zeros(6)
Sc = np.random.rand(6,6)
print(Sc)
# print(cs.mtimes(Sc,Sc.T))
# print(Sc @ Sc.T)
print(Sc[3:,:3])

[[5.17012759e-01 8.63025331e-01 5.46347427e-01 1.02706410e-01
  3.80682377e-01 3.41980726e-01]
 [8.89806935e-02 3.12473227e-01 6.42236809e-01 8.42444325e-01
  6.84379506e-01 5.50187349e-01]
 [5.43444016e-02 5.98487861e-01 4.08155531e-01 9.23948755e-01
  1.77184452e-01 4.44683209e-01]
 [6.79317285e-01 5.50480103e-01 3.11178194e-01 9.98037623e-01
  8.71218521e-01 7.63010129e-01]
 [3.82503206e-01 5.79215755e-01 4.95746457e-01 8.80434427e-01
  1.64284763e-01 1.89719799e-01]
 [6.64495479e-01 2.81804420e-01 3.54977984e-04 7.64985407e-01
  6.97814026e-01 6.58903383e-01]]
[[6.79317285e-01 5.50480103e-01 3.11178194e-01]
 [3.82503206e-01 5.79215755e-01 4.95746457e-01]
 [6.64495479e-01 2.81804420e-01 3.54977984e-04]]


In [351]:
I = linkParametric.I_parametric(); 
I.ixx = 1
I.iyy = 2
I.izz = 3

I_float = np.zeros([3,3])
I_float[0,0] = 1 
I_float[1,1] = 2 
I_float[2,2] = 3

a = utils.spatial_inertial_with_parameter(I,1,[0,0,0], [0,0,0])
b = utils.spatial_inertia(I_float, 1,[0,0,0], [0,0,0])

print(a)
print(b)

@1=1, @2=0, 
[[@1, 00, 00, @2, @2, @2], 
 [00, @1, 00, @2, @2, @2], 
 [00, 00, @1, @2, @2, @2], 
 [@2, @2, @2, @1, @2, @2], 
 [@2, @2, @2, @2, 2, @2], 
 [@2, @2, @2, @2, @2, 3]]
[[ 1.  0.  0.  0.  0. -0.]
 [ 0.  1.  0. -0.  0.  0.]
 [ 0.  0.  1.  0. -0.  0.]
 [ 0. -0.  0.  1.  0.  0.]
 [ 0.  0. -0.  0.  2.  0.]
 [-0.  0.  0.  0.  0.  3.]]
